### Task 0 - Data ingestion

In [ ]:
# installera nödvändiga paket till labben
# duckdb = databasen vi kör sql med
# pandas = få resultat som tabeller (dataframes)
%pip install duckdb pandas

In [ ]:
# importerar bibliotek
import duckdb
import pandas as pd 
from pathlib import Path 

# gå upp en nivå från notebooken (sql/ → projektrot) (fick hjälp av ai)
base_dir = Path("..").resolve()

# filvägar (motsvarar duckdb_path = "data/sakila.duckdb")
duckdb_path = base_dir / "data" / "sakila.duckdb"
load_sql_path = base_dir / "sql" / "load_sakila.sql"

print("duckdb:", duckdb_path)
print("load script:", load_sql_path)

In [ ]:
# setup
# radera eventuell gammal databas
duckdb_path.unlink(missing_ok=True)

# kör scriptet som laddar in Sakila-data (fick hjälp av ai)
with duckdb.connect(duckdb_path) as conn, open(load_sql_path, encoding="utf-8") as f:
    conn.sql(f.read())
   
   # testfråga 
    test = conn.sql("from film limit 5;").df()

test

### Task 1 - EDA in python

In [ ]:
import duckdb
from pathlib import Path


# här ansluter vi enbart till den redan skapade duckdb-databasen.
# hitta duckdb-filen (notebook ligger i sql/, därför ..)  (fick hjälp av ai)
duckdb_path = Path("..").resolve() / "data" / "sakila.duckdb"

# öppna databasen för task 1
conn = duckdb.connect(duckdb_path)

# test
conn.sql("from film limit 3;").df()



In [ ]:
# a) Filmer längre än 3 timmar

conn.sql("""
select title, length
from film
where length > 180
order by length desc;""").df()

In [ ]:
# b) Filmer med love i titeln

conn.sql("""
select title
from film
where lower(title) like '%love%' 
order by title;""").df()

In [ ]:
# c) Kortast, genomsnittlig, median och längsta film

conn.sql("""
select
    min(length) as shortest,
    avg(length) as average,
    median(length) as median,
    max(length) as longest
from film;""").df()

In [ ]:
# d) 10 dyraste filmerna per dag
# rental_rate / rental_duration
conn.sql("""
select
    title,
    rental_rate,
    rental_duration,
    round(rental_rate / rental_duration, 2) as daily_cost
from film
where rental_duration > 0
order by daily_cost desc
limit 10;""").df()

In [ ]:
# e) Topp 10 skådespelare med flest filmer

# denna fråga räknar hur många filmer varje skådespelare har medverkat i
# och visar de 10 mest aktiva skådespelarna i databasen.

conn.sql("""
select 
    concat(a.first_name, ' ', a.last_name) as actor_name,
    count(*) as movie_count
from actor as a
join film_actor as fa 
    on a.actor_id = fa.actor_id
group by a.actor_id, a.first_name, a.last_name
order by movie_count desc
limit 10;
""").df()

In [ ]:
# f) 1. Antal filmer per rating
# denna fråga visar hur många filmer som finns i varje åldersklassificering (rating)


conn.sql("""
select rating, count(*) as film_count
from film
group by rating
order by film_count desc""").df()


In [ ]:
# f) 2. Topp 10 filmkategorier med flest filmer

conn.sql("""
select c.name as category, count (*) as film_count
from category c
join film_category fc on c.category_id = fc.category_id
group by category
order by film_count desc limit 10""").df()

In [ ]:
# f) 3. Topp 10 kunder med flest rentals

conn.sql("""
select
  c.first_name || ' ' || c.last_name as customer_name,
  count(*) as rentals
from customer c
join rental r on c.customer_id = r.customer_id
group by customer_name
order by rentals desc
limit 10
""").df()

### Task 2 - Graphs

In [ ]:
# 2a) topp 5 kunder

query_top_customers = """
select
    c.customer_id,
    c.first_name || ' ' || c.last_name as customer,
    sum(p.amount) as total_spend
from customer c
join payment p
    on p.customer_id = c.customer_id
group by c.customer_id, customer
order by total_spend desc
limit 5;"""

# kör sql -> dataframe
top_customers = conn.sql(query_top_customers).df()

# bar-chart

ax = top_customers.plot(
    kind="barh",
    x="customer",
    y="total_spend",
    title="top 5 customers by total spend",
    xlabel="total spend",
    ylabel="customer",)

ax.invert_yaxis()


In [ ]:
# 2b) top film-kategorier

query_category_revenue = """
select
    c.name as category,
    sum(p.amount) as total_revenue
from category c
join film_category fc
    on c.category_id = fc.category_id
join inventory i
    on fc.film_id = i.film_id
join rental r
    on i.inventory_id = r.inventory_id
join payment p
    on r.rental_id = p.rental_id
group by c.name
order by total_revenue desc;"""

# kör sql -> dataframe
category_revenue = conn.sql(query_category_revenue).df()


# bar-chart

ax = category_revenue.plot(
    kind="barh",
    x="category",
    y="total_revenue",
    title="Total revenue per film category",
    xlabel="total revenue",
    ylabel="category",)

ax.invert_yaxis()